<a href="https://colab.research.google.com/github/ehzawad/adversarialsomething/blob/main/Cycleganstuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install scipy==1.2.1

     |████████████████████████████████| 24.8 MB 1.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [5]:
!git clone https://github.com/ehzawad/adversarial-lesions.git

Cloning into 'adversarial-lesions'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 447 (delta 140), reused 165 (delta 68), pack-reused 200
Receiving objects: 100% (447/447), 69.41 MiB | 28.82 MiB/s, done.
Resolving deltas: 100% (249/249), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!cp -r /content/drive/MyDrive/dataset/ /content/dataset

In [7]:
%cd /content/adversarial-lesions/

/content/adversarial-lesions


In [8]:
!python /content/adversarial-lesions/isic2016_scripts/data_preprocess_isic2016.py

Training data...
900it [00:46, 19.35it/s]
(900, 256, 256, 3) (900, 2)
Done!
Test data...
379it [00:17, 21.26it/s]
(379, 256, 256, 3) (379, 2)
Done!


In [9]:
!python /content/adversarial-lesions/isic2016_scripts/data_preprocess_gan.py

900it [01:17, 11.65it/s]
Done!


In [10]:
!ls -al /content/dataset

total 24
drwx------ 5 root root 4096 Aug 26 13:36 .
drwxr-xr-x 1 root root 4096 Aug 26 13:27 ..
drwx------ 4 root root 4096 Aug 26 13:32 isic2016
drwxr-xr-x 4 root root 4096 Aug 26 13:36 isic2016gan
drwxr-xr-x 2 root root 4096 Aug 26 13:35 isic2016numpy


In [11]:
!ls -al /content/dataset/isic2016gan

total 32
drwxr-xr-x 4 root root  4096 Aug 26 13:36 .
drwx------ 5 root root  4096 Aug 26 13:36 ..
drwxr-xr-x 2 root root 20480 Aug 26 13:38 trainA
drwxr-xr-x 2 root root  4096 Aug 26 13:38 trainB


In [12]:
!mkdir /content/dataset/isic2016gan/testA

In [13]:
!mkdir /content/dataset/isic2016gan/testB

In [14]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-vbbnnois
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-vbbnnois
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=446aa037b77022ecdc1b763cb6572d77df9207de3127e03661939bf8b5544427
  Stored in directory: /tmp/pip-ephem-wheel-cache-61tczk1b/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [13]:
!pip install keras==2.3.1

     |████████████████████████████████| 377 kB 5.1 MB/s 
     |████████████████████████████████| 50 kB 6.9 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires keras~=2.6, but you have keras 2.3.1 which is incompatible.


In [15]:
import os
import tensorflow as tf
import scipy
from glob import glob
import numpy as np

print(tf.test.gpu_device_name())

base_path = os.path.abspath("../")
dataset_path = os.path.join(base_path, "dataset/isic2016gan")
model_path = os.path.join(base_path, "models")
print(os.listdir(dataset_path))


class DataLoader():
    def __init__(self, dataset_name, img_res=(256, 256)): #128
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('{}/dataset/%s/%s/*'.format(base_path) % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        # rescale to [-1, 1]
        imgs = np.array(imgs)/127.5 - 1.
        
        # rescale to [0, 1]
        #imgs = imgs.astype('float32')
        #imgs = np.array(imgs)
        #imgs = imgs / 255.
        
        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = glob('{}/dataset/%s/%sA/*'.format(base_path) % (self.dataset_name, data_type))
        path_B = glob('{}/dataset/%s/%sB/*'.format(base_path) % (self.dataset_name, data_type))

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            # rescale to [-1, 1]
            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            # rescale to [0, 1]
            
            #imgs_A = imgs_A.astype('float32')
            #imgs_A = np.array(imgs_A)
            #imgs_A = imgs_A / 255.
            
            #imgs_B = imgs_B.astype('float32')
            #imgs_B = np.array(imgs_B)
            #imgs_B = imgs_B / 255.
            
            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        
        # rescale to [-1, 1]
        img = img/127.5 - 1.
        
        # rescale to [0, 1]
        #img = img.astype('float32')
        #img = img / 255.
        
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float) # np.float

/device:GPU:0
['testA', 'trainB', 'testB', 'trainA']


In [16]:
#Sanity check
path = glob('{}/dataset/%s/%s/*'.format(base_path) % ("isic2016gan", "trainB"))
path[:3]


['/content/dataset/isic2016gan/trainB/24.jpeg',
 '/content/dataset/isic2016gan/trainB/187.jpeg',
 '/content/dataset/isic2016gan/trainB/184.jpeg']

In [17]:
# Run this block once to install these libs

!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install scipy==1.2.1
!pip install pillow

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-va4sxaz4
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-va4sxaz4


In [19]:
from __future__ import print_function, division
import scipy
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import scipy.misc

import numpy as np
import os
import keras
import pandas as pd
import time

class CycleGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 256 
        self.img_cols = 256 
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'isic2016gan' #contains trainA, trainB, testA, testB
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 32
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0                    # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle    # Identity loss

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        reconstr_A, reconstr_B,
                                        img_A_id, img_B_id ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                            loss_weights=[  1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id ],
                            optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)

        # Upsampling
        u1 = deconv2d(d4, d3, self.gf*4)
        u2 = deconv2d(u1, d2, self.gf*2)
        u3 = deconv2d(u2, d1, self.gf)

        u4 = UpSampling2D(size=2)(u3)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        
        # Make a log file
        record_df = pd.DataFrame(columns=['epoch', 'd_Loss', 'accuracy', 'g_loss', 'adv', 'recon', 'id', 'elapsed_time'])

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                        [valid, valid,
                                                        imgs_A, imgs_B,
                                                        imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                #print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                #                                                        % ( epoch, epochs,
                #                                                            batch_i, self.data_loader.n_batches,
                #                                                            d_loss[0], 100*d_loss[1],
                #                                                            g_loss[0],
                #                                                            np.mean(g_loss[1:3]),
                #                                                            np.mean(g_loss[3:5]),
                #                                                            np.mean(g_loss[5:6]),
                #                                                            elapsed_time))

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
            
            
            # Print updates
            print(epoch,"--------", d_loss[0], g_loss[0], 100*d_loss[1])

            # Log metrics at end of epoch            
            new_row = {'epoch': epoch, 'd_Loss': d_loss[0], 'accuracy': 100*d_loss[1], 'g_loss': g_loss[0], 'adv': np.mean(g_loss[1:3]), 'recon': np.mean(g_loss[3:5]), 'id': np.mean(g_loss[5:6]), 'elapsed_time': elapsed_time}
                                      
            record_df = record_df.append(new_row, ignore_index=True)
            record_df.to_csv("{}/record.csv".format(model_path), index=0)
            
            # Save file at end of epoch.
            print("Saving model at {} epoch.".format(epoch))
            self.g_AB.save(filepath='{}/{}'.format(model_path, "b2m.h5"))
            keras.callbacks.ModelCheckpoint(filepath='{}/{}'.format(model_path, "b2m.h5"), verbose=1,save_best_only=True)

            self.g_BA.save(filepath='{}/{}'.format(model_path, "m2b.h5"))
            keras.callbacks.ModelCheckpoint(filepath='{}/{}'.format(base_path, "m2b.h5"), verbose=1,save_best_only=True)

            self.combined.save(filepath='{}/{}'.format(model_path, "model.h5"))
            keras.callbacks.ModelCheckpoint(filepath='{}/{}'.format(model_path, "model.h5"), verbose=1,save_best_only=True)   
            
        
        print("Training finished...")
        print("Models Saved!")


    def sample_images(self, epoch, batch_i):
        os.makedirs('{}/images/%s'.format(model_path) % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)

        # Demo (for GIF)
        
        #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("{}/images/%s/%d_%d.png".format(model_path) % (self.dataset_name, epoch, batch_i))
        plt.close()


if __name__ == '__main__':
    
    start_time = time.time()
    
    gan = CycleGAN()
    print("Training..")
    gan.train(epochs=2, batch_size=1, sample_interval=500)
    
    end_time = time.time()
    print("--- Time taken to train : %s hours ---" % ((end_time - start_time)//3600))


Training..


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:109: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resi

0 -------- 0.4882449582219124 3.521923780441284 11.23046875
Saving model at 0 epoch.
1 -------- 0.40983689576387405 5.327047348022461 5.2734375
Saving model at 1 epoch.
Training finished...
Models Saved!
--- Time taken to train : 0.0 hours ---


In [12]:
from __future__ import print_function, division
import scipy
from keras.models import load_model
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
from tqdm import tqdm
import keras
import pandas as pd
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import cv2

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)  


# Training file directory
DATASET = os.path.join(ROOT_DIR, 'dataset')

PATH = "{}/{}".format(DATASET, "isic2016numpy")
# load data
x_train = np.load("{}/x_train.npy".format(PATH))
y_train = np.load("{}/y_train.npy".format(PATH))
x_train.shape, y_train.shape

MODEL_PATH = os.path.join(ROOT_DIR, "models")
print(ROOT_DIR)
print(os.listdir(MODEL_PATH))

/content
['model.h5', 'images', 'record.csv', 'b2m.h5', 'm2b.h5']


In [2]:
!rm -rf sample_data/